In [1]:
import arcpy
import equations

In [2]:
arcpy.env.workspace = "D:\CSULB\SpeedHumps\SpeedHump.gdb"
fc = "SpeedHump\SpeedHump"
edit = arcpy.da.Editor(arcpy.env.workspace)

## Accident History Flag

In [14]:
fields = ["TEYA", "FYA", "THYA", "AH"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[3] = equations.acc_hist_flag(row[0], row[1], row[2])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

## Pre Score

In [4]:
fields = ["THYA", "FYA", "DA", "SR", "BDR", "Park", "OSF", "SWI", "SL", "SW", "CDP", "PS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        sf_int = []
        for sf in row[2:7]:
            sf_int.append(1 if sf == "Yes" else 0)

        sw = True if row[9] == "Yes" else False
        cdp = True if row[10] == "Yes" else False
        row[11] = float(equations.pre_score(row[0], row[1], sf_int, row[7], row[8], sw, cdp))
        print(row[11])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

33.9
11.5
6.9
7.0
11.2


## Speed Calculation

In [3]:
fields = ["FN_E_S", "FS_W_S", "SN_E_S", "SS_W_S", "CL85"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = equations.calc_speed(row[0], row[1], row[2], row[3])
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

24.829999923706055
32.619998931884766
32.8799991607666
29.704999923706055
33.000000953674316


## ADT Calculation

In [3]:
fields = ["FN_E_T", "FS_W_T", "SN_E_T", "SS_W_T", "CLADT"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        row[4] = equations.calc_adt(row[0], row[1], row[2], row[3])
        print(row[4])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

8308
2477
2597
7210
2320


## Rank Score

In [3]:
fields = ["SP", "CL85", "CLADT", "KSI", "FYA", "SW", "CDP", "DA", "SR", "BDR", "Park", "OSF", "SWI", "RS"]

edit.startEditing(False, True)
edit.startOperation()

with arcpy.da.UpdateCursor(fc, fields) as cursor:
    for row in cursor:
        width_factor = 1 if row[12] < 36 else 0
        sw_factor = 0.5 if row[5] == "No" else 0
        cd_factor = 0.5 if row[6] == "Yes" else 0
        
        sf_int = []
        for sf in row[8:12]:
            sf_int.append(1 if sf == "Yes" else 0)
        sf_points = equations.calc_sf_points(sf_int, width_factor)

        row[13] = equations.rank_score(row[0], row[1], row[2], row[3], row[4], sw_factor + cd_factor, row[7], sf_points)
        print(row[13])
        cursor.updateRow(row)

edit.stopOperation()
edit.stopEditing(True)

0.0
0.0
62.632491760253906
0.0
0.0
